In [26]:
# !pip install torch
# !pip install stable-baselines3
# !pip install numpy==1.20.3
# 在这里，感叹号（!）用于表示您希望在Jupyter Notebook或IPython环境中
# 运行一个命令行命令。
# 这里的命令是pip install stable-baselines3，
# 用于安装stable-baselines3库。
# 如果您在Jupyter Notebook或IPython中运行此命令，
# 它将安装库，而无需离开Python环境。

Looking in indexes: https://mirror.baidu.com/pypi/simple
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 174.5/174.5 kB 3.4 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 10.4 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 8.0 MB/s eta 0:00:00a 0:00:01
  Preparing metadata (setup.py) ... done
  Using cached https://mirror.baidu.com/pypi/packages/5d/22/f55638bea4af17edf23e1c919ad5d256141bbeec0196c450be9785f1dcb6/matplotlib-3.7.1-cp38-cp38-manylinux_2_12_x86_64.manylinux2010_x86_64.whl (9.2 MB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 11.0 MB/s eta 0:00:0000:0100:01
  Using cached https://mirror.baidu.com/pypi/packages/6c/10/a7d0fa5baea8fe7b50f448ab742f26f52b80bfca85ac2be9d35cdd9a3246/pyparsing-3.0.9-py3-none-any.whl (98 kB)
  Using cached https://mirror.baidu.com/pypi/packages/5c/f9/695d6bedebd747e5eb0fe8fad57b72fdf25411273a39791cde838d5a8f51/cycler-0.11.0-py3-none-any.whl (6.4 kB)
  Using cached htt

In [5]:
import gym 
import random

In [14]:
env = gym.make('CartPole-v1')
states = env.observation_space.shape[0]
actions = env.action_space.n

In [15]:
env = gym.make('CartPole-v1')
env


<TimeLimit<OrderEnforcing<PassiveEnvChecker<CartPoleEnv<CartPole-v1>>>>>

In [9]:
actions


2

创建一个环境

In [19]:
# Test Random Environment with OpenAI Gym

episodes = 10
for episode in range(1, episodes+1):
    state = env.reset()
    done = False
    score = 0 
    
    while not done:
        env.render()
        action = random.choice([0,1])
        n_state, reward, done, info, temp = env.step(action)
        score+=reward
    print('Episode:{} Score:{}'.format(episode, score))

Episode:1 Score:24.0
Episode:2 Score:13.0
Episode:3 Score:14.0
Episode:4 Score:19.0
Episode:5 Score:33.0
Episode:6 Score:20.0
Episode:7 Score:40.0
Episode:8 Score:22.0
Episode:9 Score:26.0
Episode:10 Score:13.0


/data/home/bofu.huang.o/conda_env/RF_env/lib/python3.8/site-packages/gym/envs/classic_control/cartpole.py:211: UserWarning: WARN: You are calling render method without specifying any render mode. You can specify the render_mode at initialization, e.g. gym("CartPole-v1", render_mode="rgb_array")
  gym.logger.warn(


In [23]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F

In [24]:
def build_model(states, actions):
    model = nn.Sequential(
        nn.Flatten(),
        nn.Linear(states, 24),
        nn.ReLU(),
        nn.Linear(24, 24),
        nn.ReLU(),
        nn.Linear(24, actions)
    )
    return model


In [ ]:
model = build_model(states, actions)

In [7]:
from stable_baselines3 import DQN
from stable_baselines3.common.env_util import make_atari_env
from stable_baselines3.common.vec_env import VecFrameStack
from stable_baselines3.dqn import MlpPolicy


In [8]:
from stable_baselines3.common.torch_layers import BaseFeaturesExtractor

class CustomNetwork(BaseFeaturesExtractor):
    def __init__(self, observation_space, features_dim=24):
        super(CustomNetwork, self).__init__(observation_space, features_dim)
        input_dim = observation_space.shape[0]
        self.fc1 = nn.Linear(input_dim, 24)
        self.fc2 = nn.Linear(24, 24)
        self.fc3 = nn.Linear(24, features_dim)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        return x

# Create the Gym environment
env = gym.make('CartPole-v0')

# Create the custom policy
custom_policy = DQN.create_policy(env, MlpPolicy, features_extractor_class=CustomNetwork)

# Create the DQN agent with the custom policy
agent = DQN(custom_policy, env, verbose=1)

# Train the agent
agent.learn(total_timesteps=50000)

# Test the trained agent
obs = env.reset()
for _ in range(1000):
    action, _ = agent.predict(obs)
    obs, reward, done, info = env.step(action)
    env.render(
    if done:
        obs = env.reset()

env.close()


AttributeError: type object 'DQN' has no attribute 'create_policy'